# Text Translation and Sentiment Analysis using Transformers

## Project Overview:

The objective of this project is to analyze the sentiment of movie reviews in three different languages - English, French, and Spanish. We have been given 30 movies, 10 in each language, along with their reviews and synopses in separate CSV files named `movie_reviews_eng.csv`, `movie_reviews_fr.csv`, and `movie_reviews_sp.csv`.

- The first step of this project is to convert the French and Spanish reviews and synopses into English. This will allow us to analyze the sentiment of all reviews in the same language. We will be using pre-trained transformers from HuggingFace to achieve this task.

- Once the translations are complete, we will create a single dataframe that contains all the movies along with their reviews, synopses, and year of release in all three languages. This dataframe will be used to perform sentiment analysis on the reviews of each movie.

- Finally, we will use pretrained transformers from HuggingFace to analyze the sentiment of each review. The sentiment analysis results will be added to the dataframe. The final dataframe will have 30 rows


The output of the project will be a CSV file with a header row that includes column names such as **Title**, **Year**, **Synopsis**, **Review**, **Review Sentiment**, and **Original Language**. The **Original Language** column will indicate the language of the review and synopsis (*en/fr/sp*) before translation. The dataframe will consist of 30 rows, with each row corresponding to a movie.

In [1]:
#! pip install diffusers transformers accelerate

#! pip install diffusers
#! conda install -c huggingface transformers==4.14.1 tokenizers==0.10.3 -y

In [2]:
import torch
from diffusers import DiffusionPipeline
from diffusers import DPMSolverMultistepScheduler
from diffusers.utils import export_to_video

pipe = DiffusionPipeline.from_pretrained("damo-vilab/text-to-video-ms-1.7b", torch_dtype=torch.float16, variant="fp16")
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload()

prompt = "Spiderman is surfing"
video_frames = pipe(prompt, num_inference_steps=25).frames
video_path = export_to_video(video_frames)

ImportError: cannot import name 'CLIPImageProcessor' from 'transformers' (C:\Users\Shmuel Naaman\Anaconda3\lib\site-packages\transformers\__init__.py)

In [3]:
# imports
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer
from transformers import pipeline
 

In [4]:
text ="La La Land est un film absolument magnifique avec des chansons qui restent dans la tête pendant des jours. Les acteurs sont incroyables et leur alchimie est palpable. Les scènes de danse sont absolument éblouissantes et l\'histoire est touchante et authentique."

fr_en_model_name = "Helsinki-NLP/opus-mt-fr-en"  
 
fr_en_model = MarianMTModel.from_pretrained(fr_en_model_name)  
fr_en_tokenizer = MarianTokenizer.from_pretrained(fr_en_model_name)
 
# encode preprocess   
inputs = fr_en_tokenizer(text, return_tensors="pt")  

# generate the translation using the model
outputs = fr_en_model.generate(**inputs) #None

# decode the generated output and return the translated text
decoded = fr_en_tokenizer.decode(outputs[0], skip_special_tokens=True)

# sentiment-analysis
model_name = "sentiment-analysis"
sentiment_classifier = pipeline(model_name)  
sentiment   = pd.DataFrame.from_records(sentiment_classifier(decoded)) 

sentiment,decoded
 

C:\Users\Shmuel Naaman\Anaconda3\lib\site-packages\transformers\generation_utils.py:1359: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 512 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


(      label     score
 0  POSITIVE  0.999891,
 'La Land is an absolutely beautiful film with songs that stay in the head for days. The actors are incredible and their alchemy is palpable. The dance scenes are absolutely dazzling and the story is touching and authentic.')

### Get data from `.csv` files and then preprocess data

In [17]:
# use the `pd.read_csv()` function to read the movie_review_*.csv files into 3 separate pandas dataframes

# Note: All the dataframes would have different column names. For testing purposes
# you should have the following column names/headers -> [Title, Year, Synopsis, Review]

def preprocess_data() -> pd.DataFrame:
    """
    Reads movie data from .csv files, map column names, add the "Original Language" column,
    and finally concatenate in one resultant dataframe called "df".
    """
    df_eng = pd.read_csv("data/movie_reviews_eng.csv")
 
    #TODO 1: Add your code here
    df_eng = df_eng.rename(columns={'Movie / TV Series':'Title', 
                                'Year':'Year', 
                                'Synopsis':'Synopsis', 
                                'Review':'Review'})
    df_eng["Original Language"]= 'eng'
    
    
    df_fr = pd.read_csv("data/movie_reviews_fr.csv")
    df_fr = df_fr.rename(columns={'Titre':'Title', 
                                'Année':'Year', 
                                'Synopsis':'Synopsis', 
                                'Critiques':'Review'})
    df_fr["Original Language"]= 'fr'
    
    df = pd.concat([df_eng, df_fr] , ignore_index=True)
    
    df_sp = pd.read_csv("data/movie_reviews_sp.csv")
    df_sp = df_sp.rename(columns={'Título':'Title', 
                                'Año':'Year', 
                                'Sinopsis':'Synopsis', 
                                'Críticas':'Review'})
    df_sp["Original Language"]= 'sp'
    
    df = pd.concat([df, df_sp] , ignore_index=True)

    return df

df = preprocess_data()

In [18]:
df['Review'][10]

'"La La Land est un film absolument magnifique avec des chansons qui restent dans la tête pendant des jours. Les acteurs sont incroyables et leur alchimie est palpable. Les scènes de danse sont absolument éblouissantes et l\'histoire est touchante et authentique."'

In [4]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language
13,Les Choristes,2004,Ce film raconte l'histoire d'un professeur de ...,"""Les Choristes est un film magnifique qui vous...",fr
22,Y tu mamá también,2001,Dos amigos adolescentes (Gael García Bernal y ...,"""Y tu mamá también es una película que se qued...",sp
29,El Incidente,2014,"En esta película de terror mexicana, un grupo ...","""El Incidente es una película aburrida y sin s...",sp
10,La La Land,2016,Cette comédie musicale raconte l'histoire d'un...,"""La La Land est un film absolument magnifique ...",fr
23,El Laberinto del Fauno,2006,"Durante la posguerra española, Ofelia (Ivana B...","""El Laberinto del Fauno es una película fascin...",sp
12,Amélie,2001,Cette comédie romantique raconte l'histoire d'...,"""Amélie est un film absolument charmant qui vo...",fr
27,El Bar,2017,Un grupo de personas quedan atrapadas en un ba...,"""El Bar es una película ridícula y sin sentido...",sp
2,Forrest Gump,1994,Forrest Gump (Tom Hanks) is a simple man with ...,"""Forrest Gump is a heartwarming and inspiratio...",eng
16,La Tour Montparnasse Infernale,2001,Deux employés de bureau incompétents se retrou...,"""Je ne peux pas croire que j'ai perdu du temps...",fr
0,The Shawshank Redemption,1994,"Andy Dufresne (Tim Robbins), a successful bank...","""The Shawshank Redemption is an inspiring tale...",eng


### Text translation

Translate the **Review** and **Synopsis** column values to English.

In [6]:
# load translation models and tokenizers
# TODO 2: Update the code below
fr_en_model_name = "Helsinki-NLP/opus-mt-fr-en"  
es_en_model_name = "Helsinki-NLP/opus-mt-es-en" 

fr_en_model = MarianMTModel.from_pretrained(fr_en_model_name) 
es_en_model = MarianMTModel.from_pretrained(es_en_model_name) 

fr_en_tokenizer = MarianTokenizer.from_pretrained(fr_en_model_name)
es_en_tokenizer = MarianTokenizer.from_pretrained(es_en_model_name)  

In [7]:
 

# TODO 3: Complete the function below
def translate(text: str, model, tokenizer) -> str:
    """
    function to translate a text using a model and tokenizer
    """
    # encode the text using the tokenizer
    inputs = tokenizer(text, return_tensors="pt") #None

    # generate the translation using the model
    outputs = model.generate(**inputs) #None

    # decode the generated output and return the translated text
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded

In [8]:
# TODO 4: Update the code below

# filter reviews in French and translate to English
fr_reviews = None
fr_reviews_en =  df[df['Original Language'] == 'fr']['Review'].apply(lambda x: translate(x, fr_en_model, fr_en_tokenizer ))#None

df.loc[df['Original Language'] == 'fr', ['Review']] = fr_reviews_en 
 

# filter synopsis in French and translate to English
fr_synopsis = None
fr_synopsis_en = df[df['Original Language'] == 'fr']['Synopsis'].apply(lambda x: translate(x,  
                                                                 fr_en_model, 
                                                                 fr_en_tokenizer ))

df.loc[df['Original Language'] == 'fr', ['Synopsis']] = fr_synopsis_en 

# filter reviews in Spanish and translate to English
es_reviews = None
es_reviews_en = df[df['Original Language']  == 'sp']['Review'].apply(lambda x: translate(x,  
                                                                 es_en_model, 
                                                                 es_en_tokenizer ))#None

df.loc[df['Original Language'] == 'sp', ['Review']] = es_reviews_en 

# filter synopsis in Spanish and translate to English
es_synopsis = None
es_synopsis_en = df[df['Original Language']  == 'sp']['Synopsis'].apply(lambda x: translate(x,  
                                                                 es_en_model, 
                                                                 es_en_tokenizer ))#None

df.loc[df['Original Language'] == 'sp', ['Synopsis']] = es_synopsis_en 

# update dataframe with translated text
# add the translated reviews and synopsis - you can overwrite the existing data

C:\Users\Shmuel Naaman\Anaconda3\lib\site-packages\transformers\generation_utils.py:1359: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 512 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
C:\Users\Shmuel Naaman\Anaconda3\lib\site-packages\transformers\generation_utils.py:1359: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 512 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
C:\Users\Shmuel Naaman\Anaconda3\lib\site-packages\transformers\generation_utils.py:1359: UserWarnin

In [9]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language
2,Forrest Gump,1994,Forrest Gump (Tom Hanks) is a simple man with ...,"""Forrest Gump is a heartwarming and inspiratio...",eng
23,El Laberinto del Fauno,2006,"During the Spanish postwar period, Ofelia (Iva...","""The Labyrinth of Fauno is a fascinating and e...",sp
28,Torrente: El brazo tonto de la ley,1998,"In this Spanish comedy, a corrupt cop (played ...","""Torrente is a vulgar and offensive film that ...",sp
16,La Tour Montparnasse Infernale,2001,Two incompetent office workers find themselves...,"""I can't believe I've wasted time watching thi...",fr
20,Roma,2018,Cleo (Yalitza Aparicio) is a young domestic wo...,"""Rome is a beautiful and moving film that pays...",sp
9,The Island,2005,In a future where people are cloned for organ ...,"""The Island is a bland and forgettable sci-fi ...",eng
3,The Godfather,1972,Don Vito Corleone (Marlon Brando) is the head ...,"""The Godfather is a classic movie that stands ...",eng
12,Amélie,2001,This romantic comedy tells the story of Amélie...,"""Amélie is an absolutely charming film that wi...",fr
13,Les Choristes,2004,This film tells the story of a music teacher w...,"""The Choristes are a beautiful film that will ...",fr
25,Águila Roja,(2009-2016),This Spanish television series follows the adv...,"""Red Eagle is a boring and uninteresting serie...",sp


### Sentiment Analysis
Use HuggingFace pretrained model for sentiment analysis of the reviews. Store the sentiment result **Positive** or **Negative** in a new column titled **Sentiment** in the dataframe.


In [8]:
# Taken from the example here in this link
# https://huggingface.co/blog/sentiment-analysis-python#2-how-to-use-pre-trained-sentiment-analysis-models-with-python
#from transformers import pipeline 
 

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [10]:
# TODO 5: Update the code below
# load sentiment analysis model
model_name = "sentiment-analysis"
sentiment_classifier = pipeline(model_name)  

# TODO 6: Complete the function below
def analyze_sentiment(text, classifier):
    """
    function to perform sentiment analysis on a text using a model
    """
    # 
    # 
    # text = df.Synopsis.tolist()

    df_t  = pd.DataFrame.from_records(classifier(text)) 
    return df_t

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

C:\Users\Shmuel Naaman\Anaconda3\lib\site-packages\huggingface_hub\file_download.py:123: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Shmuel Naaman\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [11]:
# TODO 7: Add code below for sentiment analysis
# perform sentiment analysis on reviews and store results in new column

df_t = analyze_sentiment( df.Review.tolist(), sentiment_classifier)

df['label'] = df_t.label
df['score'] = df_t.score

In [12]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language,label,score
3,The Godfather,1972,Don Vito Corleone (Marlon Brando) is the head ...,"""The Godfather is a classic movie that stands ...",eng,POSITIVE,0.999884
2,Forrest Gump,1994,Forrest Gump (Tom Hanks) is a simple man with ...,"""Forrest Gump is a heartwarming and inspiratio...",eng,POSITIVE,0.999885
29,El Incidente,2014,"In this Mexican horror film, a group of people...","""The Incident is a boring and frightless film ...",sp,NEGATIVE,0.999810
8,Solo: A Star Wars Story,2018,A young Han Solo (Alden Ehrenreich) joins a gr...,"""Dull and pointless, with none of the magic of...",eng,NEGATIVE,0.999799
4,Inception,2010,Dom Cobb (Leonardo DiCaprio) is a skilled thie...,"""Inception is a mind-bending and visually stun...",eng,POSITIVE,0.999885
5,Blade Runner 2049,2017,"Officer K (Ryan Gosling), a new blade runner f...","""Boring and too long. Nothing like the origina...",eng,NEGATIVE,0.999675
23,El Laberinto del Fauno,2006,"During the Spanish postwar period, Ofelia (Iva...","""The Labyrinth of Fauno is a fascinating and e...",sp,POSITIVE,0.999874
19,Babylon A.D.,2008,"In the distant future, a mercenary has to esco...","""This film is a complete mess. The characters ...",fr,NEGATIVE,0.999814
22,Y tu mamá también,2001,Two teenage friends (Gael García Bernal and Di...,"""And your mom is also a movie that stays with ...",sp,POSITIVE,0.999874
13,Les Choristes,2004,This film tells the story of a music teacher w...,"""The Choristes are a beautiful film that will ...",fr,POSITIVE,0.999893


In [39]:
# export the results to a .csv file
#df.to_csv("result/reviews_with_sentiment.csv", index=False)